In [1]:
import sys
import os
import numpy as np
from encode import Chromagram
from encode import Encoder


In [2]:
data_dir = "/home/dofo/Desktop/neuromorphicsnn/chord_snn/dataset/chords"

all_files = [os.path.join(root, fname)
             for root, dirs, files in os.walk(data_dir)
             for fname in files if fname.endswith('.wav')]

In [15]:
def parse_filename(filename):
    base_name = os.path.basename(filename)
    parts = base_name.split('_')
    chord_name = parts[1] +  "_" + parts[2]
    root_note_name = parts[3]
    root_midi_number = parts[4]
    return chord_name, root_note_name, root_midi_number